In [1]:
import tensorflow as tf
import os

import pandas as pd
import nltk
import spacy

from spacy.lang.pt.examples import sentences

# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
nltk.download("rslp")
# !python -m spacy download pt_core_news_sm
nlp = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/henricobela/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
# df_palavras_input_fp = pd.DataFrame(['amigo', 'amizade', 'estudante', 'esquema','carnaval','carnavalesco', 'cavaleira', 'carreiras', 'tempestade'], columns=['original_fp'])
df_palavras_input_fp = pd.DataFrame(
    [
        "amigos",
        "amigas",
        "amizade",
        "carreira",
        "carreiras",
        "tempestade",
        "cavaleira",
        "irmão",
        "irmã",
        "carro",
        "carroça",
    ],
    columns=["original_fp"],
)
df_palavras_input_fp

original_fp
0       amigos
1       amigas
2      amizade
3     carreira
4    carreiras
5   tempestade
6    cavaleira
7        irmão
8         irmã
9        carro
10     carroça

In [4]:
stemmer = nltk.stem.RSLPStemmer()

In [5]:
df_palavras_input_fp["nltk_stemmer"] = [
    stemmer.stem(palavra) for palavra in df_palavras_input_fp["original_fp"]
]
df_palavras_input_fp

original_fp nltk_stemmer
0       amigos         amig
1       amigas         amig
2      amizade       amizad
3     carreira         carr
4    carreiras         carr
5   tempestade    tempestad
6    cavaleira        caval
7        irmão          irm
8         irmã         irmã
9        carro         carr
10     carroça       carroç

In [6]:
doc = nlp(str([palavra for palavra in df_palavras_input_fp["original_fp"]]))

In [7]:
doc

['amigos', 'amigas', 'amizade', 'carreira', 'carreiras', 'tempestade', 'cavaleira', 'irmão', 'irmã', 'carro', 'carroça']

In [8]:
df_palavras_input_fp

original_fp nltk_stemmer
0       amigos         amig
1       amigas         amig
2      amizade       amizad
3     carreira         carr
4    carreiras         carr
5   tempestade    tempestad
6    cavaleira        caval
7        irmão          irm
8         irmã         irmã
9        carro         carr
10     carroça       carroç

In [9]:
df_palavras_input_fp["spacy_lemma"] = [
    token.lemma_ for token in doc if token.pos_ == "NOUN"
]
df_palavras_input_fp

ValueError: Length of values (10) does not match length of index (11)

In [10]:
sent1 = "Eu quero ter aulas na faculdade aprender e ter aprendizado enquanto posso comprar um carro caro"
sent2 = "Eu quero ir para aula com o carro do meu caro amigo e ver a aula aprender conteúdos felizes que promovam minha aprendizagem"
tokens1 = [t.lower() for t in sent1.split(" ")]
tokens2 = [t.lower() for t in sent2.split(" ")]

In [11]:
tokens1

['eu',
 'quero',
 'ter',
 'aulas',
 'na',
 'faculdade',
 'aprender',
 'e',
 'ter',
 'aprendizado',
 'enquanto',
 'posso',
 'comprar',
 'um',
 'carro',
 'caro']

In [12]:
tokens2

['eu',
 'quero',
 'ir',
 'para',
 'aula',
 'com',
 'o',
 'carro',
 'do',
 'meu',
 'caro',
 'amigo',
 'e',
 'ver',
 'a',
 'aula',
 'aprender',
 'conteúdos',
 'felizes',
 'que',
 'promovam',
 'minha',
 'aprendizagem']

In [13]:
set1 = set(tokens1)
set2 = set(tokens2)
set_union = set1.union(set2)
set_intersection = set1.intersection(set2)
print(set_intersection)
print(set_union)

{'e', 'caro', 'aprender', 'eu', 'quero', 'carro'}
{'para', 'meu', 'conteúdos', 'e', 'posso', 'aprender', 'aulas', 'ver', 'aprendizado', 'amigo', 'enquanto', 'do', 'ir', 'aprendizagem', 'comprar', 'que', 'faculdade', 'aula', 'ter', 'na', 'a', 'promovam', 'eu', 'felizes', 'caro', 'com', 'o', 'minha', 'um', 'carro', 'quero'}


In [14]:
nlp = spacy.load("pt_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Apple está querendo comprar uma startup do Reino Unido por 100 milhões de dólares
Apple PROPN nsubj
está AUX aux
querendo VERB ROOT
comprar VERB xcomp
uma DET det
startup NOUN obj
do ADP case
Reino PROPN nmod
Unido PROPN flat:name
por ADP case
100 NUM obl
milhões NUM flat
de ADP case
dólares NOUN nmod


In [15]:
def preprocess(sentence):
    result = []
    tokens = nlp(sentence)
    for token in tokens:
        # remove determiners and punctuation or spaces
        if token.pos_ in ["PUNCT", "SPACE", "DET"]:
            continue
        else:
            # the lemma of pronouns is stored as '-PRON-' in spacy; we want the actual pronoun
            if token.pos_ == "PRON":
                result.append(token.text.lower())
            else:
                result.append(token.lemma_)
    return result


print("preprocessed sent1: {}".format(preprocess("Eu gosto de estudar")))
print("preprocessed sent2: {}".format(preprocess(sent2)))

preprocessed sent1: ['eu', 'gostar', 'de', 'estudar']
preprocessed sent2: ['eu', 'querer', 'ir', 'para', 'aula', 'com', 'carro', 'de o', 'caro', 'amigo', 'e', 'ver', 'aula', 'aprender', 'conteúdo', 'feliz', 'que', 'promor', 'aprendizagem']


In [16]:
lemma_set1 = preprocess(sent1)

lemma_set2 = preprocess(sent2)

In [17]:
set1_lemma = set(lemma_set1)
set2_lemma = set(lemma_set2)
set_union_lemma = set1.union(set2_lemma)
set_intersection_lemma = set1_lemma.intersection(set2_lemma)

In [18]:
print(
    "Jaccard Coefficient (depois do lemma): {}".format(
        len(set_intersection_lemma) / len(set_union_lemma)
    )
)

Jaccard Coefficient (depois do lemma): 0.25


In [21]:
def jaccard_coefficient(sent1, sent2, preprocessor=preprocess):
    tokens1 = preprocessor(sent1)
    tokens2 = preprocessor(sent2)
    set1 = set(tokens1)
    set2 = set(tokens2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

print(jaccard_coefficient(sent1,sent2))

0.28


In [22]:
respostas = """Modelos 
Utilização
Tokenizacao
GPT
DB como contexto em NLP
Análise de sentimento
NLP com áudio
Tokenization
Geração de NL
Classificação
Como funciona GPT
Caso de uso real com NLP
Deploy de modelos
Arquitetura de redes NLP
Bibliotecas
Código
Girias
Utilizar áudio para inter
Manter conversa entre IA
Técnicas
Tratamentos
Infraestrutura
Arquitetura
Qualidade de metadados
Qualidade de conceitos de
Calculo por trás do nlp
Bag of words
Aprofundamento em NLP
Saudável
Aprender em um ritmob
Salles"""